# 臺灣道路車流追蹤
    主要是使用YOLOv8模型，與官方 model.predict 和 model.track 功能。
    
    ※ model.track 功能 : 這是YOLOv8的追蹤模式，進行物件追蹤。
    
    ※ model.predict 功能 : 這是YOLOv8的預測模式，可以從 YOLOv8 模型中，提取自訂項目的輸出，也可以自訂各種設定和參數。


## 使用套件

In [345]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

## 載入模型


In [346]:
# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

### 只想辨識某些類別? 
### 你可以參考這個連結，是coco128 的物件類別編碼表 或是 coco128.yaml檔案 (已在資料夾中)
    https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco128.yaml

In [347]:
# 2: car 
# 5:bus 
# 7:truck
# classes = [2]  #只偵測car
classes = [2, 5, 7] #只偵測car、bus、truck

In [348]:
# Set the confidence threshold 
# 設定可信度的閾值
# 大於0.3就打框
conf_thresh = 0.3

In [349]:
# Open the video file
# 開啟想要辨識的影片路徑
video_path = "./KHheighway_video/721836327.118740.mp4"

In [350]:
# 開啟影片
# 建立 VideoCapture ，讀取影片中的每一帧
cap = cv2.VideoCapture(video_path)

### 目前 YOLOv8 在使用 model.predict 的情況下，
### 可以設定"依類別過濾結果，即classes=0，或classes=[0,2,3]"
    相關使用格式，你可以參考此連結:
    https://docs.ultralytics.com/modes/predict/#inference-arguments

In [351]:
# 這個程式碼是使用 model.predict
# 儲存 predict 結果

#results = model.predict(source=video_path, save=True, classes=classes, conf=conf_thresh)

# 不要儲存 predict 結果
results = model.predict(source=video_path, save=False, classes=classes, conf=conf_thresh)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/1784) c:\Users\a0936\OneDrive\\w10-50-yolov5-\KHheighway_video\721836327.118740.mp4: 384x640 2 cars, 1 truck, 111.1ms
video 1/1 (2/1784) c:\Users\a0936\OneDrive\\w10-50-yolov5-\KHheighway_video\721836327.118740.mp4: 384x640 2 cars, 1 truck, 105.8ms
video 1/1 (3/1784) c:\Users\a0936\OneDrive\\w10-50-yolov5-\KHheighway_video\721836327.118740.mp4: 384x640 2 cars, 1 truck, 105.5ms
video 1/1 (4/1784) c:\Users\a0936\OneDrive\\w10-50-yolov5-\KHheig

    ※ model.predict: 這是 YOLOv8 模型的預測方法，用於進行目標偵測。

    ※ source=video_path: 要進行偵測影片的路徑。 video_path 是包含視訊檔案路徑的變數。

    ※ save=True: 這個參數表示是否儲存偵測結果。 設定為 True 時，偵測結果會儲存在預設目錄中(runs\detect\predict)。

    ※ classes=classes: 這個參數用來指定要偵測的目標類別。 

    ※ conf=conf_thresh: 這個參數用來設定可信度的閾值。 只有當偵測到的目標的可信度大於此閾值時，才會考慮該目標。 

### 定義計數線的兩個端點的座標
    定義計數線的座標，[x1, y1, x2, y2]，此處表示一條水平的計數線。

In [352]:
# 北向
Nouth_lines_coordinates = [300, 420, 630, 420]  # Nouth_lines_coordinates[0] is one point and Nouth_lines_coordinates[2] is another point

# 南向
South_lines_coordinates = [700, 500, 1050, 500]  # South_lines_coordinates[0] is one point and South_lines_coordinates[2] is another point

In [353]:
# Store the track history
# 使用 defaultdict 創建 track_history 字典，以追蹤每個目標的中心點坐標歷史記錄。
track_history = defaultdict(lambda: [])
crossed_line = defaultdict(lambda: 0)  # 將 crossed_line 放在這裡

# 計數南北向的車流
Nouth_totalCount = 0
South_totalCount = 0

# 初始化已訪問過的目標 ID 列表
visited_id_list = [] 

In [354]:
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    # 使用 cap.read() 讀取影格。
    # 使用 YOLOv8 模型的 track 方法進行目標追蹤
    # success 表示是否成功讀取，frame 是讀取到的影格。
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        # persist=True 持續追蹤；指定的追踪器配置文件 'bytetrack.yaml'
        results = model.track(frame, persist=True, tracker = 'bytetrack.yaml') 
        
        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu() # 目標框座標
        track_ids = results[0].boxes.id.int().cpu().tolist() # 取得追蹤結果中的目標追蹤 ID ，可以在影片中看到位於最前面id=1..

        # Visualize the results on the frame
        annotated_frame = results[0].plot() # 繪製物件中包含的所有預測類型（框、關鍵點、機率等）
        
        # Plot the tracks
        # 對每個目標的追蹤歷史記錄進行更新，並在當前影格上畫出軌跡。
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            
            # 在北向的影像上畫三條計數線BGR
            cv2.line(annotated_frame, (Nouth_lines_coordinates[0], Nouth_lines_coordinates[1]-30), (Nouth_lines_coordinates[2], Nouth_lines_coordinates[3]-30), (255, 0, 255), thickness=10)
            cv2.line(annotated_frame, (Nouth_lines_coordinates[0], Nouth_lines_coordinates[1]), (Nouth_lines_coordinates[2], Nouth_lines_coordinates[3]), (0, 0, 255), thickness=10)
            cv2.line(annotated_frame, (Nouth_lines_coordinates[0], Nouth_lines_coordinates[1]+30), (Nouth_lines_coordinates[2], Nouth_lines_coordinates[3]+30), (0, 255, 255), thickness=10)

            # 在南向的影像上畫三條計數線BGR
            cv2.line(annotated_frame, (South_lines_coordinates[0], South_lines_coordinates[1]-30), (South_lines_coordinates[2], South_lines_coordinates[3]-30), (155, 100, 255), thickness=10)
            cv2.line(annotated_frame, (South_lines_coordinates[0], South_lines_coordinates[1]), (South_lines_coordinates[2], South_lines_coordinates[3]), (0, 0, 255), thickness=10)
            cv2.line(annotated_frame, (South_lines_coordinates[0], South_lines_coordinates[1]+30), (South_lines_coordinates[2], South_lines_coordinates[3]+30), (100, 155, 255), thickness=10)


            # Draw the tracking lines 追蹤線(白線)
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
            
           
            # Draw a blue center point in the middle of the bounding box
            # 畫出藍點
            center_x, center_y = int(x + w / 16), int(y + h / 16)
            cv2.circle(annotated_frame, (center_x, center_y), 5, (255, 0, 0), -1)

            
            # 如果"北向目標的藍點"碰觸到"計數線"，則進行以下操作。
            if Nouth_lines_coordinates[0] < center_x < Nouth_lines_coordinates[2] and Nouth_lines_coordinates[1] - 30 < center_y < Nouth_lines_coordinates[
                1] + 30:
                # 如果目標的 ID 還未被訪問，則增加計數，標記已訪問的 ID，並在影像上畫一條綠色的計數線。
                if track_id not in visited_id_list:  # To count the number only once
                    Nouth_totalCount = Nouth_totalCount + 1
                    visited_id_list.append(track_id)
                    cv2.line(annotated_frame, (Nouth_lines_coordinates[0], Nouth_lines_coordinates[1]),
                            (Nouth_lines_coordinates[2], Nouth_lines_coordinates[3]),
                            (0, 255, 0), thickness=2)
                     
                    

            #如果南向目標的中心點碰觸到計數線，則進行以下操作。
            if South_lines_coordinates[0] < center_x < South_lines_coordinates[2] and South_lines_coordinates[1] - 30 < center_y < South_lines_coordinates[
                1] + 30:
                #如果目標的 ID 還未被訪問，則增加計數，標記已訪問的 ID，並在影像上畫一條綠色的計數線。
                if track_id not in visited_id_list:  # To count the number only once
                    South_totalCount = South_totalCount + 1
                    visited_id_list.append(track_id)
                    cv2.line(annotated_frame, (South_lines_coordinates[0], South_lines_coordinates[1]),
                            (South_lines_coordinates[2], South_lines_coordinates[3]),
                            (0, 255, 0), thickness=2)

             #在影像上顯示計數值
            cv2.putText(annotated_frame, f"Nouth_Total Count: {Nouth_totalCount}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"South_Total Count: {South_totalCount}", (900, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0 , 0), 2)
            #cvzone.putTextRect(img, f'Count {totalCount}', (40, 40), colorT=(255, 255, 255), colorR=(0, 0, 0))

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 car, 1 truck, 110.6ms
Speed: 16.7ms preprocess, 110.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 133.4ms
Speed: 4.0ms preprocess, 133.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 124.0ms
Speed: 4.2ms preprocess, 124.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 117.3ms
Speed: 0.0ms preprocess, 117.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 123.9ms
Speed: 12.5ms preprocess, 123.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 114.4ms
Speed: 4.2ms preprocess, 114.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 147.2ms
Speed: 0.0ms preprocess, 147.2ms inference, 8.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 98.3ms
Speed: 5.1ms preprocess, 98.3ms in

### 補充上面程式碼的說明
        results = model.track(frame, persist=True, tracker = 'bytetrack.yaml') 
        
        你可以指定追蹤器的文件，官方提供了 'bytetrack.yaml' 和 'botsort.yaml'

    更多使用的方法，你可以參考連結：https://docs.ultralytics.com/modes/track/#available-trackers

### bytetrack.yaml 內容說明

    這段設定檔中包含了一些 YOLO 追蹤器（ByteTrack）的設定參數。 讓我們逐一解釋每個參數的作用：

        1. `tracker_type: bytetrack`: 指定使用的追蹤器類型，這裡是 ByteTrack。

        2. `track_high_thresh: 0.5`: 第一次關聯的閾值。 用於確定是否將新偵測的目標與現有追蹤目標相關聯的閾值。

        3. `track_low_thresh: 0.1`: 第二次關聯的閾值。 在第一次關聯失敗後，嘗試進行第二次關聯的閾值。

        4. `new_track_thresh: 0.6`: 如果偵測未符合任何追蹤目標，則初始化新追蹤的閾值。

        5. `track_buffer: 30`: 緩衝區大小，用來計算何時移除追蹤目標。

        6. `match_thresh: 0.8`: 追蹤目標匹配的閾值。

        7. `min_box_area: 10`: 最小框面積的閾值。 用於追蹤器評估，但目前未使用。

        8. `mot20: False`: 用於追蹤器評估的標誌，目前未使用。

    這些參數的調整可以影響目標追蹤的效能和行為。 在使用 YOLO 模型進行目標偵測後，這些設定用於確定如何將偵測到的目標與先前影格中的追蹤目標相關聯。

### botsort.yaml 內容說明

    這是用於設定 BoT-SORT（Bounding box Tracker with SORT）目標追蹤器的設定參數。 讓我們逐一解釋每個參數的作用：

        1. `tracker_type: botsort`: 指定使用的追蹤器類型，這裡是 BoT-SORT。

        2. `track_high_thresh: 0.5`: 第一次關聯的閾值。 用於確定是否將新偵測的目標與現有追蹤目標相關聯的閾值。

        3. `track_low_thresh: 0.1`: 第二次關聯的閾值。 在第一次關聯失敗後，嘗試進行第二次關聯的閾值。

        4. `new_track_thresh: 0.6`: 如果偵測未符合任何追蹤目標，則初始化新追蹤的閾值。

        5. `track_buffer: 30`: 緩衝區大小，用來計算何時移除追蹤目標。

        6. `match_thresh: 0.8`: 追蹤目標匹配的閾值。

        7. `min_box_area: 10`: 最小框面積的閾值。 用於追蹤器評估，但目前未使用。

        8. `mot20: False`: 用於追蹤器評估的標誌，目前未使用。

        9. `gmc_method: sparseOptFlow`: 全域運動補償的方法。 這裡使用了 sparseOptFlow，表示使用稀疏光流進行全域運動補償。

        10. `proximity_thresh: 0.5`: 用於 ReID 模型的近距離閾值，但目前未支援。

        11. `appearance_thresh: 0.25`: 用於 ReID 模型的外觀閾值，但目前未支援。

        12. `with_reid: False`: 是否啟用 ReID 模型。 這裡是禁用的，因為設定為 `False`。

    這些參數的調整可以影響目標追蹤的效能和行為，具體效果可能需要根據具體應用場景進行偵錯和最佳化。

## 一鍵完工 (程式碼)

    from collections import defaultdict
    import cv2
    import numpy as np
    from ultralytics import YOLO

    # Load the YOLOv8 model
    model = YOLO('yolov8n.pt')

    # Open the video file
    video_path = "./KHheighway_video/721836327.118740.mp4"
    cap = cv2.VideoCapture(video_path)

    # 定義計數線的兩個端點的座標，定義計數線的座標，[x1, y1, x2, y2]，此處表示一條水平的計數線。
    # 北向
    Nouth_lines_coordinates = [300, 420, 630, 420]  # Nouth_lines_coordinates[0] is one point and Nouth_lines_coordinates[2] is another point
    # 南向
    South_lines_coordinates = [700, 500, 1050, 500]  # South_lines_coordinates[0] is one point and South_lines_coordinates[2] is another point

    # Store the track history
    # 使用 defaultdict 創建 track_history 字典，以追蹤每個目標的中心點坐標歷史記錄。
    track_history = defaultdict(lambda: [])
    crossed_line = defaultdict(lambda: 0)  # 將 crossed_line 放在這裡

    # Nouth_lines_coordinates[1] is height from origin and Nouth_lines_coordinates[2] is height from origin
    Nouth_totalCount = 0
    South_totalCount = 0
    visited_id_list = [] # 初始化已訪問過的目標 ID 列表

    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        # 使用 cap.read() 讀取影格。
        # 使用 YOLOv8 模型的 track 方法進行目標追蹤
        # success 表示是否成功讀取，frame 是讀取到的影格。
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            
            # 參考範例修改 results = model.track(source='./KHheighway_video/721836327.118740.mp4',show=True, tracker="botsort.yaml")
            results = model.track(frame, persist=True, classes=[0] , tracker = 'bytetrack.yaml') # persist=True 持續追蹤；指定的追踪器配置文件 'bytetrack.yaml'
            
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu() #目標框座標
            track_ids = results[0].boxes.id.int().cpu().tolist() # 取得追蹤結果中的目標追蹤 ID 最前面1~200...不重複

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            
            # Plot the tracks
            # 對每個目標的追蹤歷史記錄進行更新，並在當前影格上畫出軌跡。
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                
                # 在北向的影像上畫三條計數線BGR
                cv2.line(annotated_frame, (Nouth_lines_coordinates[0], Nouth_lines_coordinates[1]-30), (Nouth_lines_coordinates[2], Nouth_lines_coordinates[3]-30), (255, 0, 255), thickness=10)
                cv2.line(annotated_frame, (Nouth_lines_coordinates[0], Nouth_lines_coordinates[1]), (Nouth_lines_coordinates[2], Nouth_lines_coordinates[3]), (0, 0, 255), thickness=10)
                cv2.line(annotated_frame, (Nouth_lines_coordinates[0], Nouth_lines_coordinates[1]+30), (Nouth_lines_coordinates[2], Nouth_lines_coordinates[3]+30), (0, 255, 255), thickness=10)

                # 在南向的影像上畫三條計數線BGR
                cv2.line(annotated_frame, (South_lines_coordinates[0], South_lines_coordinates[1]-30), (South_lines_coordinates[2], South_lines_coordinates[3]-30), (155, 100, 255), thickness=10)
                cv2.line(annotated_frame, (South_lines_coordinates[0], South_lines_coordinates[1]), (South_lines_coordinates[2], South_lines_coordinates[3]), (0, 0, 255), thickness=10)
                cv2.line(annotated_frame, (South_lines_coordinates[0], South_lines_coordinates[1]+30), (South_lines_coordinates[2], South_lines_coordinates[3]+30), (100, 155, 255), thickness=10)


                # Draw the tracking lines 追蹤線(白線)
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
                
            
                # Draw a blue center point in the middle of the bounding box
                # 畫出藍點
                center_x, center_y = int(x + w / 16), int(y + h / 16)
                cv2.circle(annotated_frame, (center_x, center_y), 5, (255, 0, 0), -1)

                
                # 如果北向目標的中心點碰觸到計數線，則進行以下操作。
                if Nouth_lines_coordinates[0] < center_x < Nouth_lines_coordinates[2] and Nouth_lines_coordinates[1] - 30 < center_y < Nouth_lines_coordinates[
                    1] + 30:
                    # 如果目標的 ID 還未被訪問，則增加計數，標記已訪問的 ID，並在影像上畫一條綠色的計數線。
                    if track_id not in visited_id_list:  # To count the number only once
                        Nouth_totalCount = Nouth_totalCount + 1
                        visited_id_list.append(track_id)
                        cv2.line(annotated_frame, (Nouth_lines_coordinates[0], Nouth_lines_coordinates[1]),
                                (Nouth_lines_coordinates[2], Nouth_lines_coordinates[3]),
                                (0, 255, 0), thickness=2)
                        
                    
                # 如果南向目標的中心點碰觸到計數線，則進行以下操作。
                if South_lines_coordinates[0] < center_x < South_lines_coordinates[2] and South_lines_coordinates[1] - 30 < center_y < South_lines_coordinates[
                    1] + 30:
                    # 如果目標的 ID 還未被訪問，則增加計數，標記已訪問的 ID，並在影像上畫一條綠色的計數線。
                    if track_id not in visited_id_list:  # To count the number only once
                        South_totalCount = South_totalCount + 1
                        visited_id_list.append(track_id)
                        cv2.line(annotated_frame, (South_lines_coordinates[0], South_lines_coordinates[1]),
                                (South_lines_coordinates[2], South_lines_coordinates[3]),
                                (0, 255, 0), thickness=2)   

                # 在影像上顯示計數值
                cv2.putText(annotated_frame, f"Nouth_Total Count: {Nouth_totalCount}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(annotated_frame, f"South_Total Count: {South_totalCount}", (900, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0 , 0), 2)
                #cvzone.putTextRect(img, f'Count {totalCount}', (40, 40), colorT=(255, 255, 255), colorR=(0, 0, 0))
            
            # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()

### 後續可以增加南北向的 car、bus、truck 的功能，計算個別的數量。

In [355]:
# Initialize car_count 尚未使用到，還沒試過
# Nouth_car_count = 0
# Nouth_bus_count = 0
# Nouth_truck_count = 0

# South_car_count = 0
# South_bus_count = 0
# South_truck_count = 0

In [356]:
# 確保北向 track_id 在範圍內
#if 0 <= track_id < len(model.model.names):
#    obj_class = model.model.names[track_id]
#    if obj_class == "car":
#        Nouth_car_count += 1
#    elif obj_class == "bus":
#        Nouth_bus_count += 1
#    elif obj_class == "truck":
#        Nouth_truck_count += 1

# 確保南向 track_id 在範圍內
#if 0 <= track_id < len(model.model.names):
#    obj_class = model.model.names[track_id]
#    if obj_class == "car":
#        South_car_count += 1
#    elif obj_class == "bus":
#        South_bus_count += 1
#    elif obj_class == "truck":
#        South_truck_count += 1

In [357]:
# 北向
#cv2.putText(annotated_frame, f"Car Count: {Nouth_car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#cv2.putText(annotated_frame, f"Bus Count: {Nouth_bus_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#cv2.putText(annotated_frame, f"Truck Count: {Nouth_truck_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# 南向      
#cv2.putText(annotated_frame, f"Car Count: {South_car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#cv2.putText(annotated_frame, f"Bus Count: {South_bus_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#cv2.putText(annotated_frame, f"Truck Count: {South_truck_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    